In [1]:
import models 
import datasets
import torch
import torchaudio
import pytorch_lightning as pl
import torchmetrics
import sklearn
import sklearn.preprocessing
import pandas as pd
import numpy as np
import os

/home/stc/rybin-as/miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = '/mnt/storage/posokhov/emo_cls/data'
train = pd.read_csv('/mnt/storage/posokhov/emo_cls/filterd_train.csv')
le = sklearn.preprocessing.LabelEncoder()
le.fit(train['EmotionMaxVote'].values)


win_length = 1000
hop_length = 500
n_mels = 128
mel = torchaudio.transforms.MelSpectrogram(hop_length=hop_length, 
                                                 n_mels=n_mels, 
                                                 win_length=win_length, 
                                                 n_fft=win_length)
# mfcc = torchaudio.transforms.MFCC(n_mfcc=40, 
#                                             log_mels=True,
#                                             melkwargs={'hop_length':hop_length, 
#                                             'n_mels':n_mels, 
#                                             'win_length':win_length, 
#                                             'n_fft':win_length})
aggregator = lambda x: torch.mean(x, axis=-1)

data = datasets.audio_dataset(data_dir=data_dir, fnames=train['video'], 
                   labels=train['EmotionMaxVote'], pad_size='mean', le=le, 
                   extractor=mel, aggregator=None, as_numpy=False)



100%|██████████| 1692/1692 [01:17<00:00, 21.96it/s]


In [3]:
train, test = torch.utils.data.random_split(data, [1000, 692])
trainloader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=True)
data.pad_size


282527

In [4]:
model = models.pl_cnn2d(y_kernel=5, n_mels=n_mel, n_input=2, stride=(128, 1), n_output=len(le.classes_))
logger = pl.loggers.tensorboard.TensorBoardLogger("audio", name="mfcc")

trainer = pl.Trainer(accelerator="gpu", devices=1, max_epochs=20, logger=logger, log_every_n_steps=1)
trainer.fit(model, trainloader, testloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: audio/mfcc
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type             | Params
------------------------------------------------
0  | conv1     | Conv2d           | 12.8 K
1  | bn1       | BatchNorm2d      | 64    
2  | pool1     | MaxPool2d        | 0     
3  | drop1     | Dropout          | 0     
4  | conv2     | Conv2d           | 3.1 K 
5  | bn2       | BatchNorm2d      | 64    
6  | pool2     | MaxPool2d        | 0     
7  | drop2     | Dropout          | 0     
8  | conv3     | Conv2d           | 6.2 K 
9  | bn3       | BatchNorm2d      | 128   
10 | pool3     | MaxPool2d        | 0     
11 | drop3     | Dropout          | 0     
12 | drop4     | Dropout          | 0     
13 | fc4       | Linear           | 3.6 K 
14 | f1_train  | F1Score          | 0     
15 | f1_val    | F1Score          | 0    

Sanity Checking: 0it [00:00, ?it/s]

/home/stc/rybin-as/miniconda3/envs/torch/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:486: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
/home/stc/rybin-as/miniconda3/envs/torch/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:01<00:01,  1.62s/it]

/mnt/storage/posokhov/emo_cls/models.py:184: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return torch.nn.functional.log_softmax(x)


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:02<00:00,  1.12s/it]f1 tensor(0.1406, device='cuda:0') acc tensor(0.1406, device='cuda:0')
                                                                           

/home/stc/rybin-as/miniconda3/envs/torch/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 2:  22%|██▏       | 12/54 [01:26<05:04,  7.25s/it, loss=1.5, v_num=0]  